## 4. Views in Databricks

A **view** = a *saved SQL query* that can be reused like a table.  
Views do not store data themselves; they are **logical query definitions** on top of tables.  

Databricks provides 3 types of views with different lifecycles and scopes.

---

### Types of Views

#### 1. Stored Views (persistent objects)  
- Saved in metastore like a table.  
- Survive cluster restarts.  

---

#### 2. Temporary Views (session-scoped)  
- Exist only during Spark session.  
- Deleted when session ends.  

| Action                         | SparkSession Behavior                                            |
| ------------------------------ | ---------------------------------------------------------------- |
| Open new notebook (no cluster) | No SparkSession                                                |
| Attach notebook to cluster     | SparkSession created (global `spark`)                          |
| Detach notebook                | Notebook loses access to `spark` (cluster keeps session alive) |
| Reattach notebook              | Notebook gets SparkSession from cluster                        |
| Restart cluster                | New SparkSession created (old one lost)                       |
| Job run                        | New SparkSession per job execution                            |

---

#### 3. Global Temporary Views (cluster-scoped)  
- Stored in a special schema **`global_temp`**.  
- Accessible across sessions within the same cluster.  

In [0]:
USE CATALOG hive_metastore;

In [0]:
SHOW TABLES;

In [0]:
-- 1. Stored View
CREATE OR REPLACE VIEW top_customers AS
SELECT id, SUM(salary) AS total_salary
FROM customers
GROUP BY id
ORDER BY total_salary DESC limit 5;

-- Query view
SELECT * FROM top_customers

In [0]:
SHOW TABLES;

In [0]:
-- 2. Temporary View
CREATE OR REPLACE TEMP VIEW temp_sales AS
SELECT * FROM sales_ctas WHERE region = 'US';

-- Query temp view (only in this session)
SELECT * FROM temp_sales;

In [0]:
SHOW TABLES;

In [0]:
-- 3. Global Temporary View
CREATE OR REPLACE GLOBAL TEMP VIEW top_cust AS
SELECT * FROM customers WHERE salary > 70000;

-- Query global temp view (must prefix with schema)
SELECT * FROM global_temp.top_cust;

In [0]:
SHOW TABLES;

In [0]:
SHOW TABLES IN global_temp;

### Comparison

| Feature                | Stored View | Temp View | Global Temp View |
|-------------------------|-------------|-----------|------------------|
| **Scope**               | Permanent   | Session   | Cluster          |
| **Persistence**         | Metastore   | Memory    | `global_temp` DB |
| **Survives Restart?**   | Yes       | No      | No (until cluster restarts) |
Visible in SHOW TABLES? | Yes (in schema) | No | Yes (in global_temp)


- **Views** provide flexible query abstraction:
  - Stored → permanent in metastore.  
  - Temp → session-only.  
  - Global temp → cluster-scoped.